### 2.6

Implement minimum edit distance algorithm.

Steps of algorithm summarized in english:

1. 

In [23]:
import numpy as np
from typing import Callable

def min_edit_dist(src: str, 
                  tgt: str, 
                  del_cost: Callable[[str], int] = lambda x: 1, 
                  ins_cost: Callable[[str], int] = lambda x: 1,
                  sub_cost: Callable[[str, str], int] = lambda x,y: 0 if x==y else 2) -> int:
    """returns minium edit distance between two strings with Levenshtein distance as default"""
    n, m = len(src), len(tgt)
    D = np.zeros((n+1,m+1))
    for i in range(n):
        D[i+1,0] = D[i,0] + del_cost(src[i])
    for j in range(m):
        D[0,j+1] = D[0,j] + ins_cost(tgt[j])
        
#     for i,j in ((a,b) for a in range(1,n+1) for b in range(1,m+1)):
#         D[i,j] = min(D[i-1,j] + del_cost(src[i-1]),
#                      D[i-1,j-1] + sub_cost(src[i-1], tgt[j-1]),
#                      D[i,j-1] + ins_cost(tgt[j-1]))
    for i,j in ((a,b) for a in range(n) for b in range(m)):
        D[i+1,j+1] = min(D[i,j+1] + del_cost(src[i]),
                         D[i,j] + sub_cost(src[i], tgt[j]),
                         D[i+1,j] + ins_cost(tgt[j]))
    return D[n,m]

In [24]:
import unittest

class TestMinEditDist(unittest.TestCase):
    
    def test_trivial_base_case(self):
        src, tgt = "", ""
        src1, tgt1 = "a", "a"
        self.assertEqual(min_edit_dist(src, tgt), 0)
        self.assertEqual(min_edit_dist(src1, tgt1), 0)
        
    def test_all_delete_case(self):
        src1, tgt1 = "a", ""
        src2, tgt2 = "ab", ""
        self.assertEqual(min_edit_dist(src1, tgt1), 1)
        self.assertEqual(min_edit_dist(src2, tgt2), 2)
        
    def test_all_ins_case(self):
        src1, tgt1 = "", "a"
        src2, tgt2 = "", "ab"
        self.assertEqual(min_edit_dist(src1, tgt1), 1)
        self.assertEqual(min_edit_dist(src2, tgt2), 2)
        
    def test_all_sub_case(self):
        src1, tgt1 = "a", "b"
        src2, tgt2 = "aa", "bb"
        self.assertEqual(min_edit_dist(src1, tgt1), 2)
        self.assertEqual(min_edit_dist(src2, tgt2), 4)
        
    def test_all(self):
        src1, tgt1 = "acegh", "bdef"
        self.assertEqual(min_edit_dist(src1, tgt1), 7)
        
test = TestMinEditDist()
test_suite = unittest.TestLoader().loadTestsFromModule(test)

unittest.TextTestRunner().run(test_suite)

.....
----------------------------------------------------------------------
Ran 5 tests in 0.008s

OK


<unittest.runner.TextTestResult run=5 errors=0 failures=0>

In [92]:
import numpy as np
from typing import Callable

def aug_min_edit_dist(src: str, 
                      tgt: str, 
                      del_cost: Callable[[str], int] = lambda x: 1, 
                      ins_cost: Callable[[str], int] = lambda x: 1,
                      sub_cost: Callable[[str, str], int] = lambda x,y: 0 if x==y else 2) -> (int,str):
    """returns augmented edit distance and alignment of strings with Levenshtein as default metric"""
    print("src: ", src, "tgt: ", tgt)
    alginment = ""
    n, m = len(src), len(tgt)
    if n == 0:
        return m, "I" * m
    if m == 0:
        return n, "D" * n
    D = np.zeros((n+1,m+1))
    A = np.chararray((n+1,m+1), unicode=True)
    A[:] = ""
    for i in range(n):
        D[i+1,0] = D[i,0] + del_cost(src[i])
    for j in range(m):
        D[0,j+1] = D[0,j] + ins_cost(tgt[j])
    for i,j in ((a,b) for a in range(n) for b in range(m)):
        cost = [D[i,j+1] + del_cost(src[i]),
                D[i,j] + sub_cost(src[i], tgt[j]),
                D[i+1,j] + ins_cost(tgt[j])]
        D[i+1,j+1] = min(cost)
        a = cost.index(min(cost))
        print("cost idx: ", a)
        print("cost: {0} at {1}".format(cost,(i,j)))
        if a == 1 and src[i] == tgt[j]:
            print("sub?? really?")
            A[i+1,j+1] = A[i,j] + "_"
        elif a == 0:
            print("prev cell: ", A[i,j+1])
            A[i+1,j+1] = A[i,j+1].join("D")
            print("this cell: ", A[i+1,j+1])
        elif a == 1:
            A[i+1,j+1] = A[i,j] + "S"
        else:
            A[i+1,j+1] = A[i+1,j] + "I"
    print(D)
    print(A)
    return D[n,m], A[n,m]

In [91]:
import unittest

class TestAugMinEditDist(unittest.TestCase):

    def test_trivial_base_case(self):
        src, tgt = "", ""
        src1, tgt1 = "a", "a"
        self.assertEqual(aug_min_edit_dist(src, tgt), (0,""))
        self.assertEqual(aug_min_edit_dist(src1, tgt1), (0,"_"))
        
    def test_delete_case(self):
        src1, tgt1 = "aa", ""
        src2, tgt2 = "ab", "a"
        self.assertEqual(aug_min_edit_dist(src1, tgt1), (2,"DD"))
        self.assertEqual(aug_min_edit_dist("ab", "a"), (1,"_D"))
        
    def test_ins_case(self):
        src1, tgt1 = "", "aa"
        src2, tgt2 = "a", "ab"
        self.assertEqual(aug_min_edit_dist(src1, tgt1), (2,"II"))
        self.assertEqual(aug_min_edit_dist(src2, tgt2), (1,"_I"))
        
    def test_all_sub_case(self):
        src1, tgt1 = "a", "b"
        src2, tgt2 = "aa", "bb"
        self.assertEqual(aug_min_edit_dist(src1, tgt1), (2, "S"))
        self.assertEqual(aug_min_edit_dist(src2, tgt2), (4, "SS"))
        
aug_test = TestAugMinEditDist()
aug_test_suite = unittest.TestLoader().loadTestsFromModule(aug_test)

unittest.TextTestRunner().run(aug_test_suite)

FFF.

src:  a tgt:  b
cost idx:  0
cost: [2.0, 2.0, 2.0] at (0, 0)
prev cell:  
this cell:  D
[[ 0.  1.]
 [ 1.  2.]]
[['' '']
 ['' 'D']]
src:  aa tgt:  
src:  ab tgt:  a
cost idx:  1
cost: [2.0, 0.0, 2.0] at (0, 0)
sub?? really?
cost idx:  0
cost: [1.0, 3.0, 3.0] at (1, 0)
prev cell:  _
this cell:  D
[[ 0.  1.]
 [ 1.  0.]
 [ 2.  1.]]
[['' '']
 ['' '_']
 ['' 'D']]
src:   tgt:  aa
src:  a tgt:  ab
cost idx:  1
cost: [2.0, 0.0, 2.0] at (0, 0)
sub?? really?
cost idx:  2
cost: [3.0, 3.0, 1.0] at (0, 1)
[[ 0.  1.  2.]
 [ 1.  0.  1.]]
[['' '' '']
 ['' '_' '_']]
src:   tgt:  
src:  a tgt:  a
cost idx:  1
cost: [2.0, 0.0, 2.0] at (0, 0)
sub?? really?
[[ 0.  1.]
 [ 1.  0.]]
[['' '']
 ['' '_']]



FAIL: test_all_sub_case (__main__.TestAugMinEditDist)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-91-294eabc748a3>", line 26, in test_all_sub_case
    self.assertEqual(aug_min_edit_dist(src1, tgt1), (2, "S"))
AssertionError: Tuples differ: (2.0, 'D') != (2, 'S')

First differing element 1:
'D'
'S'

- (2.0, 'D')
?   --   ^

+ (2, 'S')
?      ^


FAIL: test_delete_case (__main__.TestAugMinEditDist)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-91-294eabc748a3>", line 15, in test_delete_case
    self.assertEqual(aug_min_edit_dist("ab", "a"), (1,"_D"))
AssertionError: Tuples differ: (1.0, 'D') != (1, '_D')

First differing element 1:
'D'
'_D'

- (1.0, 'D')
?   --

+ (1, '_D')
?      +


FAIL: test_ins_case (__main__.TestAugMinEditDist)
----------------------------------------------------------------------
Traceback (most re

<unittest.runner.TextTestResult run=4 errors=0 failures=3>